In [1]:
!pip install git+https://github.com/Flofega/NNucleate.git

  Cloning https://github.com/Flofega/NNucleate.git to /tmp/pip-req-build-it45fd2b
  Running command git clone -q https://github.com/Flofega/NNucleate.git /tmp/pip-req-build-it45fd2b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.2 MB 4.5 MB/s 
     |████████████████████████████████| 3.4 MB 58.7 MB/s 
     |████████████████████████████████| 2.3 MB 40.2 MB/s 
     |████████████████████████████████| 2.0 MB 72.9 MB/s 
     |████████████████████████████████| 334 kB 69.7 MB/s 
  Created wheel for NNucleate: filename=NNucleate-0.1-py3-none-any.whl size=10986 sha256=5d463813dde3f39fd41a73efa806f62833858b8a2878cb8b375c3fdc85472191
  Stored in directory: /tmp/pip-ephem-wheel-cache-6xovyack/wheels/44/6d/1b/ae654aec4ee2888c1de63303dd425d120ea950760fd6f9c48a
  Created wheel for MDAnalysis: filename=MDAnalysis-2.0.0-cp37-cp37m-linux_x86_64.whl size=4486237 sha256=5e85c50481f2d6cfb

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
path = "drive/MyDrive/Permutational_Invariance/"

In [ ]:
from NNucleate.trainig import test, train_perm, NNCV, CVTrajectory
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from time import time
import numpy as np

# Train multiple models with train perm and plot performance against n_trans
# Get a cpu or GPU for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))


ns = [1, 2, 5]

traj_path = path + "trajectories/dump.3Dcolloid.13.xyz"
cv_path = path + "trajectories/cn.13.dat"
top_path = path + "trajectories/reference_13.pdb"
val_path = path + "even_shuffled_6k.xtc"
val_cv_path = path + "even_shuffled_6k_cv.dat"

ds = CVTrajectory(cv_path, traj_path, top_path, 3, 92.83)
ds_val = CVTrajectory(val_cv_path, val_path, top_path, 1, 92.83)
train_set, test_set = torch.utils.data.random_split(ds, [int(len(ds)*0.8)+1, int(len(ds)*0.2)])

train_dataloader = DataLoader(train_set, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=64, shuffle=True)
val_loader = DataLoader(ds_val, batch_size=64, shuffle=True)

epochs = 20
learning_rate = 1e-4
loss_fn = nn.MSELoss()
n_at = 421

cost = []
performance = []
trainings = []
tests = []
vals = []

for n in ns:
    t1 = time()
    model = NNCV(n_at*3, 512, 256, 128).to(device)    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    #scheduler = ExponentialLR(optimizer, gamma=0.9)
    train_errors = []
    test_errors = []
    for t in range(epochs):
        print(f"Epoch {t+1}\n ------------------------------")
        tr = train_perm(train_dataloader, model, loss_fn, optimizer, n, device, 50000)
        train_errors.append(tr)
        #scheduler.step()
        te = test(test_dataloader, model, loss_fn, device)
        test_errors.append(te)

    t2 = time() - t1
    cost.append(t2)
    trainings.append(train_errors[-1])
    tests.append(test_errors[-1])
    print("---Validation---")
    val = test(val_loader, model, loss_fn, device)
    performance.append(val)

    print(" n = %d Done!" % n)

print("--- cost ---")
print(cost)
print("--- performance ---")
print(performance)
print("--- trainings ---")
print(trainings)
print("--- test ---")
print(tests)

cost = np.array(cost)/3600

fig, ax1 = plt.subplots()

ax1.plot(ns, trainings, label="Train Error")
ax1.plot(ns, tests, label="Test Error")
ax1.plot(ns, performance, label="Val. Error")
ax1.legend()
ax1.set_xlabel("Number of Permutations")
ax1.set_ylabel("RMSE")
ax1.tick_params(axis="y")

ax2 = ax1.twinx()
color = 'tab:purple'
ax2.set_ylabel("GPU hours", color=color)
ax2.plot(ns, cost, color=color, linestyle="dashed")
ax2.tick_params(axis="y", labelcolor=color)
fig.tight_layout()
plt.savefig("comp_graph.png", dpi=500)

Using cpu device


/usr/local/lib/python3.7/dist-packages/mdtraj/formats/pdb/pdbfile.py:194: RuntimeWarning: divide by zero encountered in true_divide
  particle_density = traj.top.n_atoms / traj.unitcell_volumes[0]
/usr/local/lib/python3.7/dist-packages/mdtraj/formats/pdb/pdbfile.py:198: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  'cell vectors.', category=UserWarning)


Epoch 1
 ------------------------------
loss: 158.239243 [    0/ 8001]
Avg. Test loss: 59.824761 

Epoch 2
 ------------------------------
loss: 100.098351 [    0/ 8001]
Avg. Test loss: 56.723105 

Epoch 3
 ------------------------------
loss: 121.184067 [    0/ 8001]
Avg. Test loss: 44.667684 

Epoch 4
 ------------------------------
loss: 60.197414 [    0/ 8001]
Avg. Test loss: 36.613376 

Epoch 5
 ------------------------------
loss: 35.331001 [    0/ 8001]
Avg. Test loss: 31.372870 

Epoch 6
 ------------------------------
loss: 70.368088 [    0/ 8001]
Avg. Test loss: 26.150815 

Epoch 7
 ------------------------------
loss: 70.583069 [    0/ 8001]
Avg. Test loss: 22.384462 

Epoch 8
 ------------------------------
loss: 24.916950 [    0/ 8001]
Avg. Test loss: 19.872624 

Epoch 9
 ------------------------------
loss: 43.394279 [    0/ 8001]
Avg. Test loss: 16.647719 

Epoch 10
 ------------------------------
loss: 49.528053 [    0/ 8001]
Avg. Test loss: 14.958069 

Epoch 11
 ------